### CIFAR10 Submission Notebook
### Megan Nichols

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Import data

In [2]:
test = pd.read_csv('../input/cifar10-mu/sample_submission.csv', dtype = str)
test.shape

(10000, 2)

In [3]:
test.head()

,filename,label
0,test_00001.png,0
1,test_00002.png,0
2,test_00003.png,0
3,test_00004.png,0
4,test_00005.png,0


In [4]:
test_path = ('../input/cifar10-mu/test_images')
print(len(os.listdir(test_path)))

10000


#### Build Data Generator

In [5]:
BATCH_SIZE = 64

test_datagen = ImageDataGenerator(rescale = 1/255)

test_loader = test_datagen.flow_from_dataframe(dataframe = test,
                                              directory = test_path,
                                              x_col = 'filename',
                                              batch_size = BATCH_SIZE,
                                              shuffle = False,
                                              class_mode = None,
                                              target_size = (32, 32))

Found 10000 validated image filenames.


#### Load Model

In [6]:
cnn = keras.models.load_model('../input/cifar10-model/cifar10_model_v01.h5')
cnn.summary()


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

#### Test Predictions

In [7]:
test_prob = cnn.predict(test_loader)
print(test_prob.shape)

2021-10-31 15:33:32.508857: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(10000, 10)


In [8]:
print(test_prob[:10,].round(2))

[[0.   0.   0.   0.   0.02 0.01 0.   0.97 0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.  ]
 [0.   0.04 0.   0.   0.   0.   0.   0.   0.   0.95]
 [0.   0.   0.   0.   0.   0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.99 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   1.   0.   0.   0.  ]
 [0.   0.   0.08 0.08 0.   0.83 0.01 0.01 0.   0.  ]
 [0.   0.   1.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.03 0.31 0.23 0.3  0.03 0.1  0.   0.  ]]


#### Prepare Submission

In [9]:
submission = pd.read_csv('../input/cifar10-mu/sample_submission.csv')
submission.head()

,filename,label
0,test_00001.png,0
1,test_00002.png,0
2,test_00003.png,0
3,test_00004.png,0
4,test_00005.png,0


In [10]:
test_pred = np.argmax(test_prob, axis = 1)
print(test_pred[:10])

[7 0 7 9 8 6 6 5 2 3]


In [11]:
submission.label = test_pred
submission.head()

,filename,label
0,test_00001.png,7
1,test_00002.png,0
2,test_00003.png,7
3,test_00004.png,9
4,test_00005.png,8


In [12]:
submission.to_csv('submission.csv', header = True, index = False)